In [121]:
import json
import pennylane as qml
import pennylane.numpy as np
import scipy

In [122]:
U_NP = [[1, 0, 0, 0], [0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 1, 0]]

def calculate_timbit(U, rho_0, rho, n_iters):
    """
    This function will return a timbit associated to the operator U and a state passed as an attribute.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        rho (numpy.tensor): A guess at the fixed point C[rho] = rho.
        n_iters (int): The number of iterations of C.

    Returns:
        (numpy.tensor): The fixed point density matrices.
    """
    dev = qml.device("default.mixed", wires=2)

    @qml.qnode(dev)
    def circuit(rho_input):
        qml.QubitDensityMatrix(rho_input,wires=[0,1])
        return qml.density_matrix(wires=[1])
    
    for _ in range(n_iters):
        Total_rho=np.kron(rho_0,rho)
        Total_rho=np.dot(U,np.dot(Total_rho,np.transpose(np.conjugate(U))))
        rho=circuit(Total_rho)
        #print(Total_rho)
        #print(rho)
    return rho

In [123]:
def apply_timbit_gate(U, rho_0, timbit):
    """
    Function that returns the output density matrix after applying a timbit gate to a state.
    The density matrix is the one associated with the first qubit.

    Args:
        U (numpy.tensor): A 2-qubit gate in matrix form.
        rho_0 (numpy.tensor): The matrix of the input density matrix.
        timbit (numpy.tensor): The timbit associated with the operator and the state.

    Returns:
        (numpy.tensor): The output density matrices.
    """
    Total_rho=np.kron(rho_0,timbit)
    Total_rho=np.dot(U,np.dot(Total_rho,np.transpose(np.conjugate(U))))
    
    dev = qml.device("default.mixed", wires=2)

    @qml.qnode(dev)
    def circuit(rho_input):
        qml.QubitDensityMatrix(rho_input,wires=[0,1])
        return qml.density_matrix(wires=[0])
    
    return circuit(Total_rho)

In [131]:
def SAT(U_f, q, rho, n_bits):
    """A timbit-based algorithm used to guess if a Boolean function ever outputs 1.

    Args:
        U_f (numpy.tensor): A multi-qubit gate in matrix form.
        q (int): Number of times we apply the Timbit gate.
        rho (numpy.tensor): An initial guess at the fixed point C[rho] = rho.
        n_bits (int): The number of bits the Boolean function is defined on.

    Returns:
        numpy.tensor: The measurement probabilities on the last wire.
    """
    dev = qml.device("default.mixed", wires=4)
    @qml.qnode(dev)
    def circuit(U_f):
        qml.Hadamard(wires=0)
        qml.Hadamard(wires=1)
        qml.Hadamard(wires=2)
        qml.QubitUnitary(U_f,wires=[0,1,2,3])
        return qml.density_matrix(wires=[3])
    
    rho_0=circuit(U_f)
    #print(rho_0)
    for _ in range(q):
        timbit=calculate_timbit(U_NP, rho_0, rho, 10)
        rho_0=apply_timbit_gate(U_NP, rho_0, timbit)
        
    dev = qml.device("default.mixed", wires=1)
    @qml.qnode(dev)
    def circuit2(rho_f):
        qml.QubitDensityMatrix(rho_f,wires=[0])
        return qml.probs(wires=[0])
    
    return circuit2(rho_0)

In [125]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    I = np.eye(2)
    X = qml.matrix(qml.PauliX(0))

    U_f = scipy.linalg.block_diag(I, X, I, I, I, I, I, I)
    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    q = json.loads(test_case_input)
    output = list(SAT(U_f, q, rho,4))

    return str(output)

def check(solution_output: str, expected_output: str) -> None:

    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)

    rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
    rho_0 = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]

    assert np.allclose(
        solution_output, expected_output, atol=0.01
    ), "Your NP-solving timbit computer isn't quite right yet!"

In [126]:
test_cases = [['1', '[0.78125, 0.21875]'], ['2', '[0.65820312, 0.34179687]']]

In [133]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1'...
Correct!
Running test case 1 with input '2'...
Correct!


In [132]:
I = np.eye(2)
X = qml.matrix(qml.PauliX(0))

U_f = scipy.linalg.block_diag(I, I, X, I, I, I, I, I)
rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
SAT(U_f, 1, rho,4)

array([0.78125, 0.21875])

In [68]:

calculate_timbit(U_NP, rho_0, rho, 5)

ValueError: shapes (8,8) and (4,4) not aligned: 8 (dim 1) != 4 (dim 0)

In [34]:

rho = [[0.6+0.j , 0.1-0.1j],[0.1+0.1j, 0.4+0.j]]
calculate_timbit(U_NP, rho_0, rho, 5)

array([[0.875+0.j, 0.   +0.j],
       [0.   +0.j, 0.125+0.j]])

In [35]:
apply_timbit_gate(U_NP, rho_0, rho_0)

array([[0.575+0.j   , 0.1  -0.075j],
       [0.1  +0.075j, 0.425+0.j   ]])

In [37]:
U_f

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [81]:
Total_rho=np.kron(rho_0,rho)

In [82]:
total=rho=np.dot(U_NP,np.dot(Total_rho,np.transpose(np.conjugate(U_NP))))

In [95]:
rho_0=np.array([[0.875+0.j, 0.   +0.j],[0.   +0.j, 0.125+0.j]])

In [96]:
dev = qml.device("default.mixed", wires=2)

@qml.qnode(dev)
def circuit(rho_input):
    qml.QubitDensityMatrix(rho_input,wires=[0,1])
    return qml.density_matrix(wires=[1])

In [97]:
circuit(total)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [98]:
Total_rho=np.kron(rho_0,rho_0)
Total_rho=np.dot(U_NP,np.dot(Total_rho,np.transpose(np.conjugate(U_NP))))

In [99]:
dev = qml.device("default.mixed", wires=2)

@qml.qnode(dev)
def circuit(rho_input):
    qml.QubitDensityMatrix(rho_input,wires=[0,1])
    return qml.density_matrix(wires=[0])

In [100]:
circuit(Total_rho)

array([[0.78125+0.j, 0.     +0.j],
       [0.     +0.j, 0.21875+0.j]])

In [128]:
json.loads('1')

1